In [1]:
!pip -q install langchain langchain_community langchain-groq tiktoken chromadb huggingface_hub transformers sentence-transformers einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.7/116.7 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip show langchain

Name: langchain
Version: 0.1.17
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, jsonpatch, langchain-community, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


## Setting up LangChain


In [3]:
from langchain.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import HuggingFaceHub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.llm import LLMChain
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq
from glob import glob
import os
import pandas as pd

## Initialize models and embeddings


In [4]:
embedding_name = 'nomic-ai/nomic-embed-text-v1.5'

In [5]:
# Load embedding
model_kwargs = {'trust_remote_code': True}
embedding = HuggingFaceEmbeddings(model_name=embedding_name, model_kwargs=model_kwargs)

modules.json:   0%|          | 0.00/255 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/140 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/69.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

In [6]:
# Initialize Large Language Model for question generation
repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
llm_question_gen = HuggingFaceHub(
    repo_id=repo_id, model_kwargs={"temperature": 0.2}
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(


In [7]:
# Initialize Large Language Model for answer generation
repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
llm_answer_gen = HuggingFaceHub(
    repo_id=repo_id, model_kwargs={"temperature": 0.2}
)

In [15]:
from google.colab import userdata
groq_api_key = userdata.get('GROQ_API_KEY')

# Initializing GROQ chat with provided API key, model name, and settings
model = ChatGroq(
            groq_api_key=groq_api_key, model_name="llama3-70b-8192",
                         temperature=0.2)

## Prompt templates

In [20]:
prompt_template_questions = """
You are a question bank generator for a question answering chatbot app.
Using the provided text below which is a section of a student manual, you are to generate all the possible questions from the students, teachers, parents, and staff that can be asked and answered within the given context.

------------
{text}
------------

Instructions:
Provide a complete list of questions that can be generated based on the context above..
Make sure the answers are exactly from the text (in verbatim) and should not be replaced with any synonyms or added with extra words.
This is for training an extractive question answering model.
Make sure that no parts of the contexts are missed.
Generate at least 30 questions in which the order of the questions match the order of appearance in the context.
Do not include questions that cannot be answered using the context.
Do not repeat any questions or generate the same question again in the list of questions.

Response format for each item:
Q: Question
A: Answer

QUESTIONS AND ANSWERS:
"""

PROMPT_QUESTIONS = PromptTemplate(
    input_variables=["text"],
    template=prompt_template_questions
)

## Load multiple and process documents

In [9]:
# Initialize documents and path
documents = []
root  = os.getcwd()

# Sort the file names
paths = sorted(glob('*.txt'))

# Copy each file to new file
for path in paths:
  loader = TextLoader(path)
  document = loader.load()
  documents.extend(document)

## Loop through each topic and generate questions

In [10]:
#splitting the text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=200)
texts = text_splitter.split_documents([documents[0]]) # Temporary: We only use the first topic

## Question Generation Chain

In [ ]:
# Initialize question generation chain
question_gen_chain = LLMChain(llm=model, prompt=PROMPT_QUESTIONS)

# Run question generation chain
response = question_gen_chain.invoke(texts)

In [23]:
print(response['text'])

Here are the generated questions and answers based on the provided text:

Q: What is embodied in the two statements "Education with a Mission" and "Witness to the Word"?
A: The University's Catholic identity.

Q: What is the aim of USC in terms of the holistic formation of the individual?
A: To enlighten the mind, hone skills, and positively transform character and behavior.

Q: What is the goal of every Carolinian?
A: To become a catalyst towards the realization of the kingdom of God, promoting truth, love, justice, freedom, peace, and grace.

Q: How are Carolinians recognizable in society?
A: By their contribution as the "light", "salt", and "yeast" of the Gospel.

Q: What are the characteristics of USC graduates?
A: Professionally competent and skilled (Scientia), having noble character and being value-driven (Virtus), and dedicated to social transformation (Devotio).

Q: What is the University of San Carlos' vision for the world?
A: A world where the darkness of sin and the night o

In [27]:
topic = 1
QA_pairs = []

sentences = response['text'].split('\n')
for sentence in sentences:
  if sentence.startswith('Q:'):
    Q = sentence[3:]
  elif sentence.startswith('A:'):
    A = sentence[3:]
    QA_pairs.append([Q,A,topic])

In [28]:
len(QA_pairs)

19

## Storing the answers

In [29]:
# Create a DataFrame from the list of question-answer pairs
qa_df = pd.DataFrame(QA_pairs, columns=["Question", "Answer", "Topic"])

# Save the DataFrame to a CSV file
qa_df.to_csv("questions_and_answers.csv", index=False)
print('Successfully saved questions and answers to CSV!')

Successfully saved questions and answers to CSV!


In [30]:
qa_df.head(30)

,Question,Answer,Topic
0,"What is embodied in the two statements ""Educat...",The University's Catholic identity.,1
1,What is the aim of USC in terms of the holisti...,"To enlighten the mind, hone skills, and positi...",1
2,What is the goal of every Carolinian?,To become a catalyst towards the realization o...,1
3,How are Carolinians recognizable in society?,"By their contribution as the ""light"", ""salt"", ...",1
4,What are the characteristics of USC graduates?,Professionally competent and skilled (Scientia...,1
5,What is the University of San Carlos' vision f...,A world where the darkness of sin and the nigh...,1
6,What is the University of San Carlos' vision f...,"A society where citizens are competent, noble ...",1
7,What is the University of San Carlos' mission?,To develop competent and socially responsible ...,1
8,What is the University of San Carlos' goal in ...,"To provide timely, relevant, and transformativ...",1
9,What are the University Core Values?,Integrity; Excellence; Commitment; Social resp...,1
